## Data Wrangling: Clean, Transform, Merge, Reshape

In [2]:
import pandas as pd

## Combining and merging data sets

### Database-style DataFrame merges

In [4]:
df1 = pd.DataFrame({'data1' : range(5, 12), 'key' : list('bbacaab')})
df2 = pd.DataFrame({'data2' : range(56, 59), 'key' : list('abd')})
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [5]:
df2

,data2,key
0,56,a
1,57,b
2,58,d


In [8]:
df1.merge(df2)

,data1,key,data2
0,5,b,57
1,6,b,57
2,11,b,57
3,7,a,56
4,9,a,56
5,10,a,56


By default, .merge() performs an [inner join](https://www.w3schools.com/sql/sql_join.asp) between the DataFrames, using the common columns as keys.

In [16]:
df1.merge(df2, how='outer')

,data1,key,data2
0,5.0,b,57.0
1,6.0,b,57.0
2,11.0,b,57.0
3,7.0,a,56.0
4,9.0,a,56.0
5,10.0,a,56.0
6,8.0,c,NaN
7,NaN,d,58.0


That means that it returns the cartesian product of the elements with common keys: if there are duplicates, it will return all the possible combinations:

In [17]:
df3 = pd.DataFrame({'data2' : range(56, 61), 'key' : list('abdbd')})
df1.merge(df3)

,data1,key,data2
0,5,b,57
1,5,b,59
2,6,b,57
3,6,b,59
4,11,b,57
5,11,b,59
6,7,a,56
7,9,a,56
8,10,a,56


If the columns to join on don't have the same name, or we want to join on the index of the DataFrames, we'll need to specify that.

In [19]:
df4 = pd.DataFrame({'data2' : range(56, 61), 'rkey' : list('abdbd')})
df1.merge(df4)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [20]:
df1.merge(df4, left_on='key', right_on='rkey')

,data1,key,data2,rkey
0,5,b,57,b
1,5,b,59,b
2,6,b,57,b
3,6,b,59,b
4,11,b,57,b
5,11,b,59,b
6,7,a,56,a
7,9,a,56,a
8,10,a,56,a


If there are two columns with the same name that we do not join on, both will get transferred to the resulting DataFrame with a suffix. We can customize these suffixes.

In [24]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer')

,data1,key_x,data2,key_y
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


In [25]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer', suffixes=['_customer', '_order'])

,data1,key_customer,data2,key_order
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


### Merging on index

In [33]:
df4.index = range(5,10)
df4

,data2,rkey
5,56,a
6,57,b
7,58,d
8,59,b
9,60,d


In [34]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [35]:
df1.merge(df4, left_on='data1', right_index=True)

,data1,key,data2,rkey
0,5,b,56,a
1,6,b,57,b
2,7,a,58,d
3,8,c,59,b
4,9,a,60,d


### Concatenating along an axis

In [39]:
pd.concat([df1, df2])

/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,data1,data2,key
0,5.0,NaN,b
1,6.0,NaN,b
2,7.0,NaN,a
3,8.0,NaN,c
4,9.0,NaN,a
5,10.0,NaN,a
6,11.0,NaN,b
0,NaN,56.0,a
1,NaN,57.0,b
2,NaN,58.0,d


In [7]:
pd.concat([df1, df2], axis=1) #Axis=1, uno al lado del otro

,data1,key,data2,key
0,5,b,56.0,a
1,6,b,57.0,b
2,7,a,58.0,d
3,8,c,NaN,NaN
4,9,a,NaN,NaN
5,10,a,NaN,NaN
6,11,b,NaN,NaN


In [41]:
pd.concat([df1, df4], axis=1)

,data1,key,data2,rkey
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,56.0,a
6,11.0,b,57.0,b
7,NaN,NaN,58.0,d
8,NaN,NaN,59.0,b
9,NaN,NaN,60.0,d


In [46]:
pd.concat([df1['data1'], df2['data2'], df3['data2']])

0     5
1     6
2     7
3     8
4     9
5    10
6    11
0    56
1    57
2    58
0    56
1    57
2    58
3    59
4    60
dtype: int64

In [48]:
pd.concat([df1['data1'], df2['data2'], df3['data2']], axis=1, keys=['x', 'y', 'z'])

,x,y,z
0,5,56.0,56.0
1,6,57.0,57.0
2,7,58.0,58.0
3,8,NaN,59.0
4,9,NaN,60.0
5,10,NaN,NaN
6,11,NaN,NaN


#### Digression

Attention! Be careful not to reassign to reserved words or functions- you will overwrite the variable.

In [50]:
pd.concat = [df1, df2] 
pd.concat([df3, df4]) 

TypeError: 'list' object is not callable

You can delete the overwritten variable, but you won't get back the original value. If it is an object or function from a module, you'll need to reload() the module, since Python doesn't load again an already imported module if you try to import it. reload() is useful also when you are actively developing your own module and want to load the latest definition of a function into memory.

In [59]:
del(pd.concat)
pd.concat([df3, df4]) 

AttributeError: module 'pandas' has no attribute 'concat'

In [60]:
import pandas as pd

pd.concat([df3, df4]) 

AttributeError: module 'pandas' has no attribute 'concat'

In [61]:
from importlib import reload

reload(pd)
pd.concat([df3, df4])

/home/dani/anaconda3/envs/master2019/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,data2,key,rkey
0,56,a,NaN
1,57,b,NaN
2,58,d,NaN
3,59,b,NaN
4,60,d,NaN
5,56,NaN,a
6,57,NaN,b
7,58,NaN,d
8,59,NaN,b
9,60,NaN,d


## Data transformation

### Removing duplicates

In [68]:
df1['key']

0    b
1    b
2    a
3    c
4    a
5    a
6    b
Name: key, dtype: object

In [67]:
df1['key'].duplicated()

0    False
1     True
2    False
3    False
4     True
5     True
6     True
Name: key, dtype: bool

In [69]:
df1['key'].drop_duplicates()

0    b
2    a
3    c
Name: key, dtype: object

In [70]:
df1.drop_duplicates(subset='key')

,data1,key
0,5,b
2,7,a
3,8,c


In [73]:
df1.drop_duplicates(subset='key', keep='last')

,data1,key
3,8,c
5,10,a
6,11,b


### Renaming axis indexes

In [76]:
df1.index = list('bhkdsdf')
df1

,data1,key
b,5,b
h,6,b
k,7,a
d,8,c
s,9,a
d,10,a
f,11,b


### Discretization and binning

In [85]:
import numpy as np

np.random.seed(42)

ages = pd.Series(np.random.randint(9,99, 50))
ages

0     60
1     23
2     80
3     69
4     29
5     91
6     95
7     83
8     83
9     96
10    32
11    11
12    30
13    61
14    10
15    96
16    38
17    46
18    10
19    72
20    68
21    29
22    41
23    84
24    66
25    30
26    97
27    57
28    67
29    50
30    68
31    88
32    23
33    70
34    70
35    55
36    70
37    59
38    63
39    72
40    11
41    59
42    15
43    29
44    81
45    47
46    26
47    12
48    97
49    68
dtype: int64

In [88]:
ages

0     60
1     23
2     80
3     69
4     29
5     91
6     95
7     83
8     83
9     96
10    32
11    11
12    30
13    61
14    10
15    96
16    38
17    46
18    10
19    72
20    68
21    29
22    41
23    84
24    66
25    30
26    97
27    57
28    67
29    50
30    68
31    88
32    23
33    70
34    70
35    55
36    70
37    59
38    63
39    72
40    11
41    59
42    15
43    29
44    81
45    47
46    26
47    12
48    97
49    68
dtype: int64

In [101]:
limits = [0, 14, 18, 35, 50, 65, 100]

pd.cut(ages, limits).value_counts()

(65, 100]    23
(18, 35]      9
(50, 65]      7
(35, 50]      5
(0, 14]       5
(14, 18]      1
dtype: int64

## String manipulation

### String object methods

In [103]:
bichos = pd.Series(np.random.choice(['Mantis Shrimp', 'Naked Mole Rat', 'Star Nosed Mole'],15))
bichos

0      Naked Mole Rat
1      Naked Mole Rat
2      Naked Mole Rat
3      Naked Mole Rat
4      Naked Mole Rat
5      Naked Mole Rat
6       Mantis Shrimp
7     Star Nosed Mole
8      Naked Mole Rat
9      Naked Mole Rat
10     Naked Mole Rat
11     Naked Mole Rat
12     Naked Mole Rat
13     Naked Mole Rat
14    Star Nosed Mole
dtype: object

In [105]:
bichos.upper()

AttributeError: 'Series' object has no attribute 'upper'

### Vectorized string functions in pandas

[Vectorized string functions in pandas](https://pandas.pydata.org/pandas-docs/stable/text.html) are grouped within the .str attribute of Series and Indexes. They have the same names as the regular Python string functions, but work on Series of strings.

In [107]:
bichos.str.upper()

0      NAKED MOLE RAT
1      NAKED MOLE RAT
2      NAKED MOLE RAT
3      NAKED MOLE RAT
4      NAKED MOLE RAT
5      NAKED MOLE RAT
6       MANTIS SHRIMP
7     STAR NOSED MOLE
8      NAKED MOLE RAT
9      NAKED MOLE RAT
10     NAKED MOLE RAT
11     NAKED MOLE RAT
12     NAKED MOLE RAT
13     NAKED MOLE RAT
14    STAR NOSED MOLE
dtype: object

In [108]:
bichos.str.lower()

0      naked mole rat
1      naked mole rat
2      naked mole rat
3      naked mole rat
4      naked mole rat
5      naked mole rat
6       mantis shrimp
7     star nosed mole
8      naked mole rat
9      naked mole rat
10     naked mole rat
11     naked mole rat
12     naked mole rat
13     naked mole rat
14    star nosed mole
dtype: object

In [109]:
bichos.str.len()

0     14
1     14
2     14
3     14
4     14
5     14
6     13
7     15
8     14
9     14
10    14
11    14
12    14
13    14
14    15
dtype: int64

In [114]:
bichos.str.split()

0      [Naked, Mole, Rat]
1      [Naked, Mole, Rat]
2      [Naked, Mole, Rat]
3      [Naked, Mole, Rat]
4      [Naked, Mole, Rat]
5      [Naked, Mole, Rat]
6        [Mantis, Shrimp]
7     [Star, Nosed, Mole]
8      [Naked, Mole, Rat]
9      [Naked, Mole, Rat]
10     [Naked, Mole, Rat]
11     [Naked, Mole, Rat]
12     [Naked, Mole, Rat]
13     [Naked, Mole, Rat]
14    [Star, Nosed, Mole]
dtype: object

In [111]:
bichos.str[:6]

0     Naked 
1     Naked 
2     Naked 
3     Naked 
4     Naked 
5     Naked 
6     Mantis
7     Star N
8     Naked 
9     Naked 
10    Naked 
11    Naked 
12    Naked 
13    Naked 
14    Star N
dtype: object

In [117]:
bichos.str.split().str[-1]

0        Rat
1        Rat
2        Rat
3        Rat
4        Rat
5        Rat
6     Shrimp
7       Mole
8        Rat
9        Rat
10       Rat
11       Rat
12       Rat
13       Rat
14      Mole
dtype: object